In [17]:
from enum import IntEnum

class _dataframe_field(IntEnum):
    original_index = 0
    line_flag = 1
    circle_flag = 2
    arc_flag = 3
    point_flag = 4
    start_x = 5
    start_y = 6
    end_x = 7
    end_y = 8
    mid_x = 9
    mid_y = 10
    angle = 11
    length = perimeter = 12
    d_x = 13
    d_y = 14
    u_x = t_x = 15
    u_y = t_y = 16
    n_x = r_x = 17
    n_y = r_y = 18
    center_x = 19
    center_y = 20
    radius = 21
    start_angle = 22
    end_angle = 23
    arc_span = 24
    
    @classmethod
    def count(cls) -> int: return len(cls)

import torch
import math

In [18]:
import ezdxf

rotation=45.; mirror_axis=None

rotation = math.radians(rotation)

dxf_file = "C:\\Users\\Rafael\\Desktop\\test.dxf"

doc = ezdxf.readfile(dxf_file)
modelSpace = doc.modelspace()

entities = [entity for entity in modelSpace if entity.dxftype() in ('LINE', 'POINT', 'CIRCLE', 'ARC')]

F = _dataframe_field
dataframe = torch.zeros((F.count(), len(entities)), dtype=torch.float32)

for i, entity in enumerate(entities):
    dataframe[F.original_index,i] = i
    entity_type = entity.dxftype()
    
    if entity_type == 'LINE':
        dataframe[F.line_flag,i] = 1
        dataframe[F.start_x,i], dataframe[F.start_y,i], _ = entity.dxf.start
        dataframe[F.end_x,i], dataframe[F.end_y,i], _ = entity.dxf.end
    
    elif entity_type == 'POINT':
        dataframe[F.point_flag,i] = 1
        dataframe[F.start_x,i], dataframe[F.start_y,i], _ = entity.dxf.location
    
    elif entity_type in ('CIRCLE', 'ARC'):
        dataframe[F.circle_flag,i] = 1 if entity_type == 'CIRCLE' else 0
        dataframe[F.arc_flag,i] = 1 if entity_type == 'ARC' else 0
        dataframe[F.radius,i] = entity.dxf.radius
        dataframe[F.center_x,i], dataframe[F.center_y,i], _ = entity.dxf.center
        dataframe[F.start_angle,i] = 0 if entity_type == 'CIRCLE' else entity.dxf.start_angle
        dataframe[F.end_angle,i] = 360 if entity_type == 'CIRCLE' else entity.dxf.end_angle

In [19]:
# Apply mirror
if mirror_axis == 'x':
    dataframe[F.start_y], dataframe[F.end_y], dataframe[F.center_y] = -dataframe[F.start_y], -dataframe[F.end_y], -dataframe[F.center_y]
    dataframe[F.start_angle], dataframe[F.end_angle] = (-dataframe[F.end_angle] % 360, -dataframe[F.start_angle] % 360)
elif mirror_axis == 'y':
    dataframe[F.start_x], dataframe[F.end_x], dataframe[F.center_x] = -dataframe[F.start_x], -dataframe[F.end_x], -dataframe[F.center_x]
    dataframe[F.start_angle], dataframe[F.end_angle] = (-(dataframe[F.end_angle] - 180) % 360, -(dataframe[F.start_angle] - 180) % 360)

In [20]:
# Rotate coordinates
rotation_sin, rotation_cos = math.sin(rotation), math.cos(rotation)

dataframe[F.start_x], dataframe[F.start_y] = (dataframe[F.start_x] * rotation_cos - dataframe[F.start_y] * rotation_sin, \
                                              dataframe[F.start_x] * rotation_sin + dataframe[F.start_y] * rotation_cos)

dataframe[F.end_x], dataframe[F.end_y] = (dataframe[F.end_x] * rotation_cos - dataframe[F.end_y] * rotation_sin, \
                                          dataframe[F.end_x] * rotation_sin + dataframe[F.end_y] * rotation_cos)

dataframe[F.center_x], dataframe[F.center_y] = (dataframe[F.center_x] * rotation_cos - dataframe[F.center_y] * rotation_sin, \
                                                dataframe[F.center_x] * rotation_sin + dataframe[F.center_y] * rotation_cos)

# Convert angles to radians and rotate
dataframe[F.start_angle] = (torch.deg2rad(dataframe[F.start_angle]) + rotation) % (2*torch.pi)
dataframe[F.end_angle] = (torch.deg2rad(dataframe[F.end_angle]) + rotation) % (2*torch.pi)

In [ ]:
mask_x = (min_x[:, None] <= max_x[None, :]) & (min_x[None, :] <= max_x[:, None])
mask_y = (min_y[:, None] <= max_y[None, :]) & (min_y[None, :] <= max_y[:, None])

mask_x = (min_x[:, newaxis] <= max_x[newaxis, :]) & (min_x[newaxis, :] <= max_x[:, newaxis])
mask_y = (min_y[:, newaxis] <= max_y[newaxis, :]) & (min_y[newaxis, :] <= max_y[:, newaxis])

mask_x = (min_x[:, torch.newaxis] <= max_x[torch.newaxis, :]) & (min_x[torch.newaxis, :] <= max_x[:, torch.newaxis])
mask_y = (min_y[:, torch.newaxis] <= max_y[torch.newaxis, :]) & (min_y[torch.newaxis, :] <= max_y[:, torch.newaxis])

In [ ]:
# Rotate coordinates
dataframe[F.start_x] = dataframe[F.start_x] * torch.cos(rotation) - dataframe[F.start_y] * torch.sin(rotation)
dataframe[F.start_y] = dataframe[F.start_x] * torch.sin(rotation) + dataframe[F.start_y] * torch.cos(rotation)
dataframe[F.end_x] = dataframe[F.end_x] * torch.cos(rotation) - dataframe[F.end_y] * torch.sin(rotation)
dataframe[F.end_y] = dataframe[F.end_x] * torch.sin(rotation) + dataframe[F.end_y] * torch.cos(rotation)
dataframe[F.center_x] = dataframe[F.center_x] * torch.cos(rotation) - dataframe[F.center_y] * torch.sin(rotation)
dataframe[F.center_y] = dataframe[F.center_x] * torch.sin(rotation) + dataframe[F.center_y] * torch.cos(rotation)

# Convert angles to radians and rotate
dataframe[F.start_angle] = (torch.deg2rad(dataframe[F.start_angle]) + rotation) % (2*torch.pi)
dataframe[F.end_angle] = (torch.deg2rad(dataframe[F.end_angle]) + rotation) % (2*torch.pi)



In [21]:
rotation=45.

start_angle = 295
end_angle = 350

start_angle, end_angle

(295, 350)

In [22]:
start_angle = (start_angle + rotation) % 360
end_angle = (end_angle + rotation) % 360

start_angle, end_angle

(340.0, 35.0)

In [ ]:
if rotation:
    

In [ ]:
x_aligned = dataframe[F.start_x] * torch.cos(rotation) - dataframe[F.start_y] * torch.sin(rotation)
y_aligned = dataframe[F.start_x] * torch.sin(rotation) + dataframe[F.start_y] * torch.cos(rotation)

In [ ]:
# 155, 283

In [3]:
# x
start_angle = 0
end_angle = 360

start_angle, end_angle = -end_angle % 360, -start_angle % 360
start_angle, end_angle

(0, 0)

In [6]:
# y
start_angle = 348
end_angle = 152

start_angle, end_angle = -(end_angle - 180) % 360, -(start_angle - 180) % 360
start_angle, end_angle 

(28, 192)

In [8]:
# y
start_angle = 108
end_angle = 249

start_angle, end_angle = (180 - end_angle) % 360, (180 - start_angle) % 360
start_angle, end_angle 

(291, 72)

In [ ]:
if mirror_axis == 'x': dataframe[F.start_y] = -dataframe[F.start_y]; dataframe[F.end_y] = -dataframe[F.end_y]
elif mirror_axis == 'y': dataframe[F.start_x] = -dataframe[F.start_x]; dataframe[F.end_x] = -dataframe[F.end_x]

In [ ]:
x_aligned = dataframe[F.start_x] * torch.cos[rotation] - dataframe[F.start_y] * torch.sin[rotation]

In [14]:
tensor1 = torch.tensor([0., 1., 2., 3.])
tensor2 = torch.tensor([4., 5., 6., 7.])
tensor3 = torch.vstack([tensor1, tensor2])

In [16]:
tensor3.long()

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])